In [1]:
# 共通の依存ライブラリ
#!conda install -y ipywidgets 
#!pip install simpleaudio
#!conda install -y pyaudio

# LocalLLMを実行する場合の依存ライブラリ
#!conda install -y pytorch torchvision torchaudio pytorch-cuda=11.7 transformers sentencepiece ipywidgets -c pytorch -c nvidia

# OpenAILLMを実行する場合の依存ライブラリ
#!conda install -y openai -c conda-forge

# ChatGPTを実行する場合の依存ライブラリ
#!git clone https://github.com/mmabrouk/chatgpt-wrapper.git
#!pip install -r chatgpt-wrapper/requirements.txt

In [9]:
import time
import re
import random
import requests
import pyaudio

from IPython.display import JSON, HTML
from ipywidgets import *

# 言語モデルクラスの定義
自分の選択肢に応じて、以下の方法のいずれかを選択する。

## (1) ローカルでLLMを実行する場合に実行

In [ ]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

class LocalLLM:
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
        self.model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")
        self._head = ""

        if torch.cuda.is_available():
            print("Using cuda")
            self.model = self.model.to("cuda")
        else:
            print("Using CPU")

    def session(self):
        pass
    
    def head(self, text):
        self._head = text
            
    def stateful(self):
        return False
    
    def generate(self, text, times, stop):
        times.append(time.time())
        token_ids = self.tokenizer.encode(self._head + text, add_special_tokens=False, return_tensors="pt")
        times.append(time.time())
        with torch.no_grad():
            output_ids = self.model.generate(
                token_ids.to(self.model.device),
                max_new_tokens=20,
                do_sample=True,
                top_k=500,
                top_p=0.95,
                pad_token_id=self.tokenizer.pad_token_id,
                bos_token_id=self.tokenizer.bos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )
        times.append(time.time())
        output = self.tokenizer.decode(output_ids.tolist()[0])
        return output
model = LocalLLM()

## (2) OpenAIのLLMを実行する場合に実行
同じディレクトリにAPI-token.txtというファイルがあり、その中にOpenAIのAPIアクセスキーが書かれていることが前提

In [30]:
import openai
class OpenAILLM:
    def __init__(self):
        openai.organization = ""
        token = open("API-token.txt", "r").read().strip();
        openai.api_key = token
        display(JSON(openai.Model.list()))
        self._head = ""

    def session(self):
        pass
        
    def head(self, text):
        self._head = text
        
    def stateful(self):
        return False
    
    def generate(self, text, times, stop, max_tokens=500):
        times.append(time.time())
        while True:
            try:
                res = openai.Completion.create(
                    model="text-davinci-003",
                    prompt=self._head + text,
                    max_tokens=max_tokens,
                    temperature=0.7,
                    stop=stop)
                output = res.choices[0].text
                times.append(time.time())
                return output
            except RateLimitError as e:
                time.sleep(10)
model = OpenAILLM()

<IPython.core.display.JSON object>

## (3) ChatGPT経由でLLMを実行する場合に実行
- 次を参照してインストールしてください。
　https://github.com/mmabrouk/chatgpt-wrapper
 
```
pip install git+https://github.com/mmabrouk/chatgpt-wrapper
playwright install firefox
chatgpt install #ブラウザが立ち上がるので、そこでChatGPTにログイン。ログイン後はWindowを消して良い。
```
- 下記の方法でバックグラウンドサービスを起動しておく

```
python chatgpthelper.py
```

In [62]:
import json
import subprocess,socket
class ChatGPTLLM:
    def __init__(self):
#        self.bot = subprocess.Popen(["python", "chatgpthelper.py"])
        pass
        
    def method(self, data):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((socket.gethostname(), 9876))

        loop = True
        response = ""
        def read():
            return s.recv(1024).decode("utf-8").split("\n")
        s.send(("%s\n---\n"%json.dumps(data)).encode('utf-8'))
        while loop:
            lines = read()
            for line in lines:
                if line == "---":
                    loop = False
                    break
                response += line
        res = json.loads(response)
        s.close()
        return res
        
    def session(self):
        data = {"method": "session", "args": []}
        self.method(data)
        
    def head(self, text):
        data = {"method": "head", "args": [text]}
        self.method(data)
        
    def generate(self, text, times, stop, max_tokens=500):
        data = {"method": "generate", "args": [text, times, stop, max_tokens]}
        return self.method(data)

    def stateful(self):
        return True

model = ChatGPTLLM()

# 声の出力
- voicevoxを使ってwavファイルを取得後、Linux上のaplayを使って直接声を出す。
- 自分の選択肢に従って次のいずれかを実行する

## (1) 音声のみを出力する場合に実行

In [64]:
# ヘルパ関数：　アルファベットをカタカナに変更
# https://qiita.com/kunishou/items/814e837cf504ce287a13

import MeCab
import unidic
import pandas as pd
import alkana
import re
import os

def alpha_to_kana(text):
    #半角英字判定
    alphaReg = re.compile(r'^[a-zA-Z]+$')
    def isalpha(s):
        return alphaReg.match(s) is not None

    sample_txt = text

    wakati = MeCab.Tagger('-Owakati')
    wakati_result = wakati.parse(sample_txt)
    #print(wakati_result)

    df = pd.DataFrame(wakati_result.split(" "),columns=["word"])
    df = df[df["word"].str.isalpha() == True]
    df["english_word"] = df["word"].apply(isalpha)
    df = df[df["english_word"] == True]
    df["katakana"] = df["word"].apply(alkana.get_kana)

    dict_rep = dict(zip(df["word"], df["katakana"]))

    for word, read in dict_rep.items():
        sample_txt = sample_txt.replace(word, read or "")
    return sample_txt

In [11]:
def play_speech(text, speaker=1):
    
    res1 = requests.post("http://localhost:50021/audio_query", params={"text": text, "speaker": speaker})
    data = res1.json()
    wav = requests.post("http://localhost:50021/synthesis", params={"speaker": speaker}, json=data)
    wav_data = wav.content
    import tempfile

    path=""
    with tempfile.NamedTemporaryFile(suffix=".wav") as f:
        path=f.name
        f.write(wav.content)
        !aplay -q {path}

## (2) inochi2dのアバターを動かすバージョン。
このノートブックの実行前に、モーションを裏で管理するサービスを別途起動しておく
```
python -m avatar_cli
```

In [65]:
import socket, json, random, math
import requests, tempfile, time
vowel_map = {
    "a": (0, 1),
    "i": (0.25, 0.5),
    "e": (0.5, 0.8),
    "o": (0.75, 0.75),
    "u": (1.0, 0.7)
}
consonant_map = {
    "k": (0.25, 0.25),
    "s": (0.5, 0.3),
    "t": (0.5, 0.5),
    "n": (0.5, 0.5),
    "h": (None, 0.6),
    "m": (None, 0),
    "y": (0.75, 0.75),
    "w": (0.75, 0.25),
    "N": (0.5, 0)
}

def make_mouth_map(w):
    mouth_map = {}
    for map in [consonant_map, vowel_map]:
        if w in map:
            for i, var in enumerate(("x", "y")):
                if map[w][i]:
                    mouth_map[f"mouth_{var}"] = map[w][0]
    return mouth_map

def interpolate(action_map, absolute_time, name, start_time, end_time, src_value, dest_value, step = 0.01):
    cur_time = start_time
    while cur_time < end_time:
        pos = (cur_time - start_time) / (end_time - start_time)
        ratio = math.sin(math.pi / 2 * pos)
        action_map.append((absolute_time + cur_time, 
                           absolute_time + cur_time + step, 
                           {name: src_value * (1 - ratio) + dest_value * ratio}))
        cur_time += step

def play_speech(text, speaker=1):
    text = alpha_to_kana(text)
    res1 = requests.post("http://localhost:50021/audio_query", params={"text": text, "speaker": speaker})
    data = res1.json()
    wav_res = requests.post("http://localhost:50021/synthesis", params={"speaker": speaker}, json=data)
    wav_data = wav_res.content
    
    path = tempfile.gettempprefix()+".wav"
    
    with open(path, "wb") as f:
        f.write(wav_data)

    action_map = []
    total_time = 0
    start_time = time.time()
    face_pitch = 0.5
    for acc in data["accent_phrases"]:
        accent_start_time = total_time
        for m in acc["moras"]:
            c = m["consonant"]
            if c:
                mouth_map = make_mouth_map(c)
                next_time = total_time + m["consonant_length"]
                action_map.append((start_time + total_time, start_time + next_time, mouth_map))
                total_time = next_time
            v = m["vowel"]
            if v:
                mouth_map = make_mouth_map(v)
                next_time = total_time + m["vowel_length"]
                action_map.append((start_time + total_time, start_time + next_time, mouth_map))
                total_time = next_time
        accent_mid_time = total_time

        dest_face_pitch = random.uniform(0.6, 0.8) if face_pitch >= 0.5 else random.uniform(0.2, 0.4)
        interpolate(action_map, start_time, "face_pitch", accent_start_time, accent_mid_time, face_pitch, dest_face_pitch)
        face_pitch = dest_face_pitch

        if acc["pause_mora"]:
            next_time = total_time + acc["pause_mora"]["vowel_length"]
            action_map.append((start_time + total_time, start_time + next_time, None))
            total_time = next_time

            accent_end_time = total_time
            dest_face_pitch = 0.5
            interpolate(action_map, start_time, "face_pitch", accent_mid_time, accent_end_time, face_pitch, dest_face_pitch)
            face_pitch = dest_face_pitch
    if face_pitch != 0.5:
        interpolate(action_map, time.time(), "face_pitch", 0, 0.2, face_pitch, 0.5)
        
    p = subprocess.Popen(["aplay", "-q", path])

    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((socket.gethostname(), 9998))
    data = json.dumps(action_map).encode('utf-8')
    s.send(len(data).to_bytes(4, 'big'))
    s.send(data)
    s.close()

    p.wait()
    os.remove(path)

In [61]:
play_speech("これは音声発話のtestなんですよ")

## GPUの利用状況確認

In [33]:
!nvidia-smi

/bin/bash: nvidia-smi: コマンドが見つかりません


# 言語モデルを使ったアバター活用アプリ実装

# 事前準備：キャラ設定とモデルの初期化

In [45]:
characters = {
    "ミク": "くだけた口調。語尾は「でしょ」「だよね」「❤」になることがある。可愛い女の子。一人称は「ボク」。ときどきネガティブになる。",
    "ユリ": "丁寧な口調。「です。」「ですよね。」をつける。おとなしい女の子。一人称は「私」。",
    "マリサ": "女子。解説役。くだけた口調。語尾には「だぜ」「なんだぜ」をつける。お調子者。一人称は「私」。",
    "レイム": "女子。聞き役。くだけた口調。語尾は「だね」「ということだね」が多い。達観して落ち着いている。一人称は「私」。",
    "ギンプたん": "可愛い女の子。丁寧な口調。少し気弱。素直で明るい。なんでも前向きに考えようとする。絵を描くのが好き。絵画にはこだわりがある"
}

# (1) チャット
人や他のAIと連動して会話をする。

## ChatPlayer

```
ChatPlayer(cname, setting):
```
- `cname`
  キャラクターの名前
- `setting`
  キャラクターの設定。喋り方、性格、今の状況や意図など。チャットに反映される。

```
ChatPlayer.response(user, input, can_silent)
```
- `user`
  直前に喋ったユーザ。ユーザ名が「ト書き」の場合、キャラクタに指示する命令として追加後に、モデルを呼ばずに終了する。
- `input`
  直前に喋ったコメント内容
- `can_silent`
  無言（コメント無視）してよいかどうか

In [73]:
class ChatPlayer:
    MAX_HISTORY_LEN=40
    def __init__(self, cname, setting):
        self.character = cname
        self.setting   = setting
        self.state     = ""
        self.history   = []
        self.buffered  = ""
        self.situation = "複数の人との会話"

    def response(self, user, input, can_silent = False):
        if user == "ト書き":
            if model.stateful():
                self.buffered += f"{user}（{input}）"
            else:
                self.history.append(f"{user}（{input}）")
            return None, None, None
        else:
            self.history.append(f"{user}「{input}」")
        silent = ""
        if can_silent:
            silent = "/ 無言の場合は【NOP】と記載する"

        rules = [
            "会話の形式で書く",
            "{self.character}はト書きで書かれた指示に従う",
            "{self.character}のコメントだけを書く", 
#            "{self.character}の感情の表現は「うれしい・起こる・悲しい・楽しい」のいずれかの内容を《》で囲んで記載する",
            "{self.character}の感情の表現は絵文字で記載する",
            "{self.character}が何かを知ったり考えた場合は台詞の後に、台詞とは別に【】書きで記載する",
            silent,
            "{self.situation}のロールプレイをしてください。",
            "{self.character}のパートを担当し、会話に続く{self.character}の台詞を１つだけ作成してください。",
            "{self.character}のパートは台詞が長く複数の段落に渡っても構いません。",
            "ただし、{self.character}の台詞は、すべて1つの「」の中に入れる必要があり、台詞の終わり以外に」を入れてはいけません。",
            "{self.character}の台詞の中に【】を入れてはいけません"
        ]
        rules_text = [ r.format(self = self) for r in rules]
        text = """>>>【ルール】
{rules}
【{character}の設定】
{setting}""".format(situation=self.situation, character=self.character, setting=self.setting + self.state, rules = " / ".join(rules_text))

        model.head(text)
        text = """
{history}
{character}「""".format(character=self.character, history=("\n".join(self.history) if not model.stateful() and len(self.history)>0 else self.history[-1] if len(self.history) else "") + self.buffered,)
        self.buffered = ""
        response = ""
        times = []
        output = model.generate(text, times, stop=[">>>"])
#        output_text = output.split("」")[0]
#        output_text = re.sub("【(.*)】", "", output_text)
#        output_text = re.sub("《(.*)》", "", output_text)
        output_text = output
        expression = None
        matched = re.match(".*【(.*)】", output)
        if matched:
            if matched[1] == "NOP":
                return None, None, None
            self.state=matched[1]
        else:
            matched = re.match(".*《(.*)》", output)
            if matched:
                expression = matched[1]
        response += output_text
        text += output_text
        self.history.append(f"{self.character}「{response}」")
        times.append(time.time())
        if len(self.history) > self.MAX_HISTORY_LEN:
            self.history = self.history[-self.MAX_HISTORY_LEN:]

        return (times[-1] - times[0]), response, expression

## 人と会話するテスト

In [74]:
cname = "ギンプたん"
user  = "私"
model.session()

miku_chat = ChatPlayer(cname, characters[cname])
miku_chat.situation = "私との部屋での会話"

# 別途AIに作成させたラップ。こう言うのを裏で指示して、結果をロールプレイに伝えられるようになると良い…

text = Text()
display(text)
out = Output()
display(out)
def handle_submit(sender):
    global history, out
    with out:
        input = text.value
        text.value = ""
        print(f"{user}：「{input}」")
        duration, res, others = miku_chat.response(user, input)
        print("%s：「%s」%s (%3.2f sec)"%(cname, res, others, duration))
        play_speech(res)

text.on_submit(handle_submit)

Text(value='')

Output()

## AI同士で会話するテスト

In [ ]:
attendees = {
 "マリサ": ChatPlayer("マリサ", characters["マリサ"]),
 "レイム": ChatPlayer("レイム", characters["レイム"])
}
model.session()

# AIに会話する際の意図を持たせる。今の所、いきなり全ての意図を喋ってしまいがちなので、どのように書くか工夫が必要
attendees["マリサ"].setting += "今は勉強をしていて明日からテストがある。どうしても数学が分からないので誰かに教えて欲しい。わからないのは三角定理。sinとcosの読み方がわからない。理系の教科はよくわからない。"
attendees["レイム"].setting += "今日はそれほど忙しくないので勉強の相手をしてあげても良い。レイムは明後日の国語のテストが分からないので教えて欲しい。"

res = "今何してるの？"; pp = "レイム"
print("%s: 「%s」"%(pp, res))
play_speech(res, 3)

exit = False
for i in range(0, 10):
    if exit:
        break
    for i, p in enumerate(["マリサ", "レイム"]):
        if exit:
            break
        duration,res,exp = attendees[p].response(pp, res, True)
        if duration is None:
            continue
        if res == "" or res == "NOP":
            exit = True
            print("---FIN---")
            break
        print("%s[%s]: 「%s」 (%s)"%(p, attendees[p].state, res, exp))
        play_speech(res, i+3)
        pp=p

# (2) 要約


In [51]:
class Summarizer:
    def __init__(self, cname, setting):
        self.character = cname
        self.setting = setting

    def summarize(self, input):
        text = """>>>【{character}の設定】
{setting}
【ニュースを500-1000文字に要約してコメント】
要約は{character}の口調で書く。{character}の感想も付けてコメントする。
【ニュース】「{news}」
【要約】""".format(character = self.character, setting = self.setting, news = input)
        response = ""
        times = []
        output = model.generate(text, times, [">>>"], 2000)
        response += output
        times.append(time.time())
        return (times[-1] - times[0]), response

## 要約のテスト

In [53]:
input = """
今月17日、打ち上げが中止された日本の新たな主力ロケット「H3」の初号機について、JAXA＝宇宙航空研究開発機構は、メインエンジンに電力を供給するロケットの1段目にある機器で異常が起きたとする調査結果を明らかにしました。JAXAは原因を究明したうえで来月10日までに再び打ち上げに臨む方針です。
「H3」の初号機は今月17日午前10時37分、鹿児島県の種子島宇宙センターから打ち上げられる予定でしたが機体の1段目にある機器が異常を検知したため、補助ロケットに着火信号を送らず打ち上げが中止されました。
JAXAは、今月18日に初号機を組み立て棟に戻したあと、原因調査を本格化させていて、22日に文部科学省の有識者会議でこれまでの調査結果を報告しました。
報告によりますと、打ち上げの6秒ほど前、メインエンジンの燃焼が始まったあと、燃焼を調整する機器に電力を供給する「VーCON1」と呼ばれる装置の内部で電流と電圧の値がゼロになる異常が発生していたことが判明したということです。
電源の異常を検知すると、補助ロケットに着火信号を送らないということでJAXAは、装置の内部にあるスイッチの動作や機器と地上設備との間の電気系統などを中心に詳しい原因を調べているということです。
また、補助ロケットを含む機体や地上設備のほか、搭載している衛星には損傷がないとして、原因を究明し対策を講じたうえで、予備の打ち上げ期間にあたる来月10日までに再び打ち上げに臨む方針です。
"""
model.session()
cname = "ミク"
miku_sum = Summarizer(cname, characters[cname])
duration, output = miku_sum.summarize(input)
print(output)
play_speech(output)

「H3」の初号機の打ち上げが中止された原因が、メインエンジンに電力を供給するロケットの1段目にある機器で異常が起きたことだとJAXAが調査結果を明らかにした。原因を究明した上で来月10日までに再び打ち上げに臨む方針となっている。報告によると、メインエンジンの燃焼が始まったあと、燃焼を調整する機器に電力を供給する「VーCON1」と呼ばれる装置の内部で電流と電圧の値がゼロになる異常が発生していたことが判明した。JAXAは装置の内部にあるスイッチの動作や機器と地上設備との間の電気系統などを中心に詳しい原因を調べているということだ。
【ミクの感想】
「H3」の初号機の打ち上げが中止された原因は、なんと機器の異常だったんだって。びっくりだね。JAXAは原因を調査中で、来月10日までに再び打ち上げに臨む予定だって。地上設備や衛星には損傷がないそうだから、良かったね。でも、やっぱりロケットの打ち上げって難しいんだなぁ。


# (3) 掛け合いで解説

In [54]:
class ManzaiPlayer:
    MAX_HISTORY_LEN=40
    def __init__(self, characters):
        self.characters = characters

    def tell(self, content):
        cs = list(self.characters.keys())
        text = ">>>"
        for k,v in self.characters.items():
            text += "【%sの設定】%s\n"%(k,v)
        text+="""
【解説内容】
{content}
【解説の仕方】１つずつ{c1}が説明、{c2}が相槌をうつ。

{c1}、{c2}のロールプレイで解説してください。""".format(content=content, c1 = cs[0], c2 = cs[1])
        response = ""
        times = [time.time()]
        output = model.generate(text, times, stop=[">>>"], max_tokens=2000)
        times.append(time.time())
        return (times[-1] - times[0]), output

## 解説のテスト

In [68]:
model.session()
player=ManzaiPlayer({"マリサ": characters["マリサ"], "レイム": characters["レイム"]})
output = player.tell("""
概要：AI画像生成ツールは、人工知能技術を利用して、様々な種類の画像を自動的に生成することができるツールです。

使い方：通常、ツールにアクセスし、必要なパラメーターを設定して、ボタンをクリックするだけで、簡単に画像を生成することができます。多くの場合、生成された画像は、サイズや解像度などの特徴をカスタマイズすることができます。

応用例：AI画像生成ツールは、デザイナー、アーティスト、マーケター、研究者など、様々な分野で活用されています。例えば、製品やブランドのプロモーション用の広告画像や、Webサイトの背景画像、仮想現実の世界の背景画像、研究に必要な画像データの生成などに利用されます。

注意点：AI画像生成ツールは、一部の制限や制約があることがあります。例えば、生成された画像が著作権侵害になる場合があること、生成された画像が人工的であることがわかる場合があることなどです。そのため、ツールを利用する前に、利用規約や注意事項を確認し、慎重に利用することが重要です。

""")
print(output[1])
play_speech(output[1])

マリサ：おーい、聞いて聞いて！AI画像生成ツールっていうのがあるんだって！
レイム：ああ、AI画像生成ツールね。それは、人工知能技術を使って、色んな種類の画像を自動で作れるんだよ。
マリサ：そうそう！パラメーターを設定して、ボタンを押すだけで、カスタマイズした画像を簡単に作れるっていうわけさ。
レイム：たしかに、デザイナーやアーティスト、マーケター、研究者など、色んな分野で使われているよね。例えば、Webサイトの背景画像や、仮想現実の世界の背景画像、広告画像などにも使われている。
マリサ：あと、研究に必要な画像データを生成するためにも使われるってことだぜ。
レイム：でも、使う前には注意が必要だよね。例えば、著作権侵害になることがあったり、人工的であることがわかる場合もあるから、利用規約や注意事項を確認して慎重に利用することが大切だよ。


# (4) [実装中] キャラ口調への変換

In [ ]:
def response2(input):
    global history
    text = """私が入力する文章をミクの設定に従ってミクの台詞に変換してください。
入力された文章を単純に変換してください。

【ミクの設定】
くだけた口調。「でしょ、だよね、❤」をつける。可愛い女の子。一人称は「ボク」。ときどきネガティブになる。

入力された文章：「{}」
ミクの台詞：「""".format(input)
    times = []
    output = model.generate(text, times)
    response = output
    return (times[-1] - times[0]), response